# JH Coronavirus Analysis

## Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import chart_studio.plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import cufflinks as cf
init_notebook_mode(connected=True)
%matplotlib inline
cf.go_offline()

## Data

In [2]:
cases_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
deaths_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

## Import data

### Cases

In [3]:
cases_df = pd.read_csv(cases_csv)
cases_df.drop(labels=['Lat','Long','Province/State'],axis=1,inplace=True)
cases_df = cases_df.groupby('Country/Region').sum()

# Transpose the data so that column names are countries and time is index
cases_df = pd.DataFrame(data=cases_df.values[:,1:].transpose(),
                              index=cases_df.columns[1:],
                              columns=cases_df.index.to_list())

# Convert time (string) to datetime
cases_df.index = pd.to_datetime(cases_df.index)

### Deaths

In [4]:
deaths_df = pd.read_csv(deaths_csv)
deaths_df.drop(labels=['Lat','Long'],axis=1,inplace=True)
deaths_df = deaths_df.groupby('Country/Region').sum()

# Transpose the data so that column names are countries and time is index
deaths_df = pd.DataFrame(data=deaths_df.values[:,1:].transpose(),
                              index=deaths_df.columns[1:],
                              columns=deaths_df.index.to_list())

# Convert time (string) to datetime
deaths_df.index = pd.to_datetime(deaths_df.index)

## Data Analysis

In [5]:
latest_date = max(cases_df.index)
latest_date

Timestamp('2020-04-19 00:00:00')

### Worlwide cases and deaths

In [6]:
world_df = pd.DataFrame()
world_df['Cases'] = cases_df.sum(axis=1)
world_df['Deaths'] = deaths_df.sum(axis=1)

world_df.iplot()

### Number of cases (top 10 countries)

In [7]:
latest_data = cases_df[cases_df.index==latest_date]
top_10_cases = latest_data.apply(pd.Series.nlargest,axis=1,n=10).columns
cases_df[top_10_cases].iplot()

### Number of deaths (top 10 countries)

In [8]:
latest_data = deaths_df[deaths_df.index==latest_date]
top_10_deaths = latest_data.apply(pd.Series.nlargest,axis=1,n=10).columns
deaths_df[top_10_deaths].iplot()

### Daily growth rate

In [9]:
cases_df[top_10_cases].pct_change().iplot()

In [10]:
cases_df[top_10_cases].diff().iplot(kind='bar')

## World map

In [11]:
latest_df = pd.DataFrame()
latest_df['Country'] = cases_df.columns
latest_df['Cases'] = cases_df[cases_df.index==latest_date].values.transpose()

In [12]:
data = dict(
        type = 'choropleth',
        locations = latest_df['Country'],
        z = latest_df['Cases'],
        text = latest_df['Country'],
        colorbar_title = 'Number of cases',
        locationmode = "country names",
        colorscale = 'Darkmint'
        )

layout = dict(
    title = 'Number of cases',
    geo = dict(
        showframe = True,
        projection = {'type':'natural earth'}
    )
)

choromap = go.Figure(data=[data],layout=layout)
iplot(choromap)